In [24]:
def chunk_list(data_list, chunk_size):
    n = max(1, chunk_size)
    return (data_list[i:i+chunk_size] for i in range(0, len(data_list), chunk_size))

with open('input.csv', 'r') as input_file:
    data = input_file.read()
    data_list = data.split()
    
    n = data_list[0]
    graph_data = chunk_list(data_list[1:], 3)

    print(data_list)
    print(n)
    print(list(graph_data))


['5', '10', '2', '2', '3', '1', '2']
5
[['10', '2', '2'], ['3', '1', '2']]


In [ ]:
['5', '10', '2', '2', '3', '1', '2']